# Objective Day 1

## Objective

Create a function to scrape the Billboards 100 HOT songs and create local dataframe of songs with them including:

Song’s name

Song’s artist

## Importing Libraries <a class="anchor" id="day1libraries"></a>

In [2]:
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import requests

## Source <a class="anchor" id="day1source"></a>

In [3]:
url = "https://www.billboard.com/charts/hot-100"

In [4]:
response = requests.get(url)
print(response.status_code) #200 is Succeess, 1xx Informational responses, 3xx is Redirection, 4xx is Client Errors, 5xx is Server errors

200


In [5]:
soup = BeautifulSoup(response.text, "html.parser")

In [6]:
#for tag in soup.select("span.chart-element__information"): #Path to the container that contains the required information. If there is a unique class name it will be enough.
#    print(tag.select("span.chart-element__information__song.text--truncate.color--primary")[0].get_text())
#    print(tag.select("span.chart-element__information__artist.text--truncate.color--secondary")[0].get_text())

## Code <a class="anchor" id="day1code"></a>


In [7]:
song_titles = [tag.select("span.chart-element__information__song.text--truncate.color--primary")[0].get_text() for tag in soup.select("span.chart-element__information")]
#song_titles
#List with song titles

In [8]:
song_artists = [elem.select("span.chart-element__information__artist.text--truncate.color--secondary")[0].get_text() for elem in soup.select("span.chart-element__information")]
#song_artists
#List with song artists

In [9]:
d = {"song_artists":song_artists, "song_titles":song_titles} #Creating a dictionary with all the information

In [10]:
df_100_charts = pd.DataFrame(data=d) #Converting the dictionary to a dataframe

In [11]:
df_100_charts

,song_artists,song_titles
0,BTS,Butter
1,Olivia Rodrigo,Good 4 U
2,Dua Lipa Featuring DaBaby,Levitating
3,Justin Bieber Featuring Daniel Caesar & Giveon,Peaches
4,Silk Sonic (Bruno Mars & Anderson .Paak),Leave The Door Open
...,...,...
95,Lainey Wilson,Things A Man Oughta Know
96,Thomas Rhett,Country Again
97,Elle King & Miranda Lambert,Drunk (And I Don't Wanna Go Home)
98,Lil Baby & Lil Durk,If You Want To


------------------

# Objective Day 2



## Objective 1

* Create a second function to retrieve the audio features of a given song.


## Importing Libraries

In [12]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from Credentials import Client_ID, Client_Secret
import json
import requests
client_credentials_manager = SpotifyClientCredentials(client_id=Client_ID, client_secret=Client_Secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
import pprint
import pandas as pd

## Audio features from one song

Create a second function to retrieve the audio features of a given song.

In [13]:
song = sp.search(q='Butter', limit=1) #Getting information about one song.

In [47]:
#song #The json with the info about the song is a dictionary

In [42]:
song.keys()

dict_keys(['tracks'])

In [43]:
song.get("tracks").keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [44]:
#song['tracks']['items'][0]

In [18]:
song['tracks']['items'][0].keys() #This dictionary contains our wanted id.

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [19]:
#pprint.pprint(song) #pprint is the pretty-printer library showing code in a prettier way.

The following code takes the information from the dict within a dict etc. until it gets the id for the one song we chose above. The [0] shows that one part is a list and the first item on that list contains our information.

In [20]:
song['tracks']['items'][0]['id']

'2bgTY4UwhfBYhGT4HUYStN'

Now we get the audiofeatures of our chosen song.

In [21]:
song["tracks"]["items"][0]["uri"] #The uri for the song.

'spotify:track:2bgTY4UwhfBYhGT4HUYStN'

In [46]:
audio_features = sp.audio_features(song["tracks"]["items"][0]["uri"])[0]
#audio_features     #The audio-features of the song are shown as a dictionary.

In [45]:
#audio_features.keys()

In [24]:
audio_features_new = { key: [audio_features[key]] for key in list(audio_features.keys()) }

In [25]:
pd.DataFrame(audio_features_new) #Dataframe with the audio features for our chosen song.

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.759,0.459,8,-5.187,1,0.0948,0.00323,0,0.0906,0.695,109.997,audio_features,2bgTY4UwhfBYhGT4HUYStN,spotify:track:2bgTY4UwhfBYhGT4HUYStN,https://api.spotify.com/v1/tracks/2bgTY4UwhfBY...,https://api.spotify.com/v1/audio-analysis/2bgT...,164442,4


--------------------------------

## Objective 2

* Create a third function to update the internal database with the song features using the previous function and to add this information to the dataframe:

    - Song’s url/ uri
    - Song’s audio_features

Recommendation: create a function to extend the internal database with songs of your choice/playlist,.... The more songs you have in the database, the better.

## Functions


In [26]:
#################### Function to get the uri of a song name ######################

def get_uri_of_song(song_name):
    results = sp.search(q=song_name, limit=1)
    uri = results["tracks"]["items"][0]["uri"]
    return uri

########### Function to get the audio features of a song as a dictionary ###########

def get_audio_features_of_song(song_name):
    audio_features = sp.audio_features(song_name["tracks"]["items"][0]["uri"])[0]
    audio_features = { key: [audio_features[key]] for key in list(audio_features.keys()) }
    return audio_features

##################### Function to get id of a song ##############################

def get_song_id(song_name):
    results = sp.search(q=song_name, limit=1)
    id = results['tracks']['items'][0]['id']
    return id


################## Function to get the songs from a playlist ####################
def get_playlist_tracks(username, playlist_id): #username is always "spotify" and playlist_id is the id in the link.
    results = sp.user_playlist_tracks(username, playlist_id)
    tracks = results["items"]
    while results["next"]:
        results = sp.next(results)
        tracks.extend(results["items"])
    return tracks


############### Function to get the artist from a track#################
def get_artists_from_track(track):
    return [artist["name"] for artist in track["artists"]]


############# Function to get the artist from playlist##############
def get_artists_from_playlist(playlist_id):
    tracks_from_playlist = get_playlist_tracks("spotify", playlist_id)
    return list(set(artist for subset in [get_artists_from_track(track["track"]) for track in tracks_from_playlist] for artist in subset))

----------------------------------

## Code

### Top 100 Songs

Getting all the song names from our Hot Songs 100 and making a dataframe with uri. 

In [57]:
#df_100_charts #The dataframe containing the 100 Songs from the webpage.

In [59]:
#song_artists

In [ ]:
#song_titles

In [60]:
top_100_uris = []
for i in song_titles:
    top_100_uris.append(get_uri_of_song(i))

In [62]:
#top_100_uris

In [71]:
top_100_uris_audio_features = []
for i in range(len(top_100_uris)):
    top_100_uris_audio_features.append(sp.audio_features(top_100_uris[i]))
    #######IMPORTANT: Make function out of audiofeature getting.

In [66]:
#top_100_uris_audio_features

In [128]:
top_100_uris_audio_features #list_with_audio_features

[[{'danceability': 0.759,
   'energy': 0.459,
   'key': 8,
   'loudness': -5.187,
   'mode': 1,
   'speechiness': 0.0948,
   'acousticness': 0.00323,
   'instrumentalness': 0,
   'liveness': 0.0906,
   'valence': 0.695,
   'tempo': 109.997,
   'type': 'audio_features',
   'id': '2bgTY4UwhfBYhGT4HUYStN',
   'uri': 'spotify:track:2bgTY4UwhfBYhGT4HUYStN',
   'track_href': 'https://api.spotify.com/v1/tracks/2bgTY4UwhfBYhGT4HUYStN',
   'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2bgTY4UwhfBYhGT4HUYStN',
   'duration_ms': 164442,
   'time_signature': 4}],
 [{'danceability': 0.556,
   'energy': 0.661,
   'key': 6,
   'loudness': -5.052,
   'mode': 0,
   'speechiness': 0.204,
   'acousticness': 0.3,
   'instrumentalness': 0,
   'liveness': 0.101,
   'valence': 0.668,
   'tempo': 168.56,
   'type': 'audio_features',
   'id': '6PERP62TejQjgHu81OHxgM',
   'uri': 'spotify:track:6PERP62TejQjgHu81OHxgM',
   'track_href': 'https://api.spotify.com/v1/tracks/6PERP62TejQjgHu81OHxgM',
   '

In [80]:
top_100_uris_audio_features[0][0] #First 0 is i, second 0 is always same.

{'danceability': 0.759,
 'energy': 0.459,
 'key': 8,
 'loudness': -5.187,
 'mode': 1,
 'speechiness': 0.0948,
 'acousticness': 0.00323,
 'instrumentalness': 0,
 'liveness': 0.0906,
 'valence': 0.695,
 'tempo': 109.997,
 'type': 'audio_features',
 'id': '2bgTY4UwhfBYhGT4HUYStN',
 'uri': 'spotify:track:2bgTY4UwhfBYhGT4HUYStN',
 'track_href': 'https://api.spotify.com/v1/tracks/2bgTY4UwhfBYhGT4HUYStN',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2bgTY4UwhfBYhGT4HUYStN',
 'duration_ms': 164442,
 'time_signature': 4}

In [113]:
#Empty lists to get out the audiofeatures for every single song.
#Feature lists (empty)
danceabilitiy = []
energy = []
key = []
loudness = []
mode = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []
type = []
id = []
uri = []
track_href = []
analysis_url = []
duration_ms = []
time_signature = []



In [115]:
single_features = list(top_100_uris_audio_features[0][0].keys())
#Accessing every single feature in the order as the feature lists above. Returns a list.
#list_with_feature_names

In [130]:
#def get_audio_feature_items_into_seperate_lists(list_with_audio_features, list_with_feature_names):

def get_audio_feature_items_into_seperate_lists(list_with_feature_names):
    for feature in list_with_feature_names:
        print(feature)
        #for i in range(len(list_with_audio_features)):
            #single_features[feature].append(list_with_audio_features[i][0][str(feature)])

#for i in range(len(top_100_uris_audio_features)):
#    danceabilitiy_.append(top_100_uris_audio_features[i][0]["danceability"])

In [131]:
get_audio_feature_items_into_seperate_lists(single_features)

danceability
energy
key
loudness
mode
speechiness
acousticness
instrumentalness
liveness
valence
tempo
type
id
uri
track_href
analysis_url
duration_ms
time_signature


### Random Playlists

Getting the information from a random playlist.

In [49]:
big_playlist_id = "4rnleEAOdmFAbRcNCgZMpY" #This playlist has around 5000 songs.
playlist =  get_playlist_tracks("spotify", big_playlist_id) #This creates a list with all the songs in the playlist.

In [50]:
#playlist[0].keys()

In [51]:
#playlist[0]["track"].keys()

In [30]:
playlist[0]["track"]["uri"] #The path to the uri for each song. 

'spotify:track:7dS5EaCoMnN7DzlpT6aRn2'

In [52]:
# This part creates two lists, one with the uri's the other with the artists names.

playlist_uri = [] #Creating a list with all the uris to get the other information.
artist_name = [] #Creating a list with all the artist names.

for i in range(len(playlist)):
    playlist_uri.append(playlist[i]["track"]["uri"]) #The i is the name of the song in the playlist.
    artist_name.append(playlist[i]["track"]["artists"][0]["name"])
    
    
    #playlist_audio_features.append(sp.audio_features(i["tracks"]["items"][0]["uri"])[0])

In [32]:
 #Creating sublists of the uri's with 100 items in each sublist(last one has 95).
sublists_uri = [playlist_uri[x:x+100] for x in range(0, len(playlist_uri), 100)]
#sp.audio_features(list of uri)

In [33]:
#For loop for every sublist to get the audio features. This creates a list with lists with dicts. Distribution--> [52x[100x{}] 1x[95x{}]]

smaller_playlists_audio_features = []
for i in range(len(sublists_uri)):
    smaller_playlists_audio_features.append(list(sp.audio_features(sublists_uri[i])))

#playlist_audio_features.append(sp.audio_features(i["tracks"]["items"][0]["uri"])[0])


In [35]:
#playlist[0]["track"]["name"]

In [55]:
#To check if the for loop above worked
#first = sp.audio_features(sublists_uri[0])
#first 

# Objective Day 3

## Importing Libraries

In [132]:
import pandas as pd
from sklearn import datasets # sklearn comes with some toy datasets to practise
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

## Functions

## Objective 1

Fit the standard scaler to scale the audio features of each song using both dataframes.

## Objective 2

Save the freshly trained standard scaler with pickle

## Objective 3

Get the user’s song audio features

## Objective 4

Apply the trained scaler to transform the user’s song audio features

## Objective 5

Create a function to fit  the K-Means clustering method using all the songs contained in the both datasets ( hot100 and spotify ).

## Remember to:

* Do some research on the optimal K-value for the K-means
* Save the final K-means model with pickle
* Use the trained K-means model to predict the cluster of each song in the internal databases and add this information to the internal databases

# Objective Day 4

## Importing Libraries

## Functions

## Objective 1

Create a function to ask the user:

Song title

Song artist


## Objective 2

Create a function to search the user song in the Spotify API to get the audio features

## Objective 3

Create a function to retrieve the standard scaler previously saved and scales the song’s audio features

## Objective 4

Create a function to retrieve the K-Means model and predicts to which cluster belongs the song.


## Objective 5

Create a function to retrieve one/more songs at random from the internal database which belongs to the same cluster as the user’s song and it’s not the same as the user song.